In [1]:
from sa import *
import os, json
import numpy as np

WEIGHT_FILE = "./qm.json"
TEST_INPUT = "./img0.npy"
RANDOM_MEM_GEN = False
DO_PRINT = False
GENERATE_ISA = False
decode = False
with open(os.path.join(WEIGHT_FILE)) as fp:
    QM_DATA = json.loads(fp.read())

ADDR_MIN = 0x0000_0000
ADDR_MAX = 0x0002_0000
OFF_MEM_WB_BASE_ADDR = 0x0000_0000
OFF_MEM_UB_BASE_ADDR = 0x0001_AA80

Q = 5   # Q-Num format (Q-2.5)

In [2]:
# Instantiation
SA = SYSTOLIC_ARRAY(gen_isa=GENERATE_ISA, USE_Q_NUMBER=True, Q=4)
OFF_MEM = BRAM(depth=8192, data_num=16, nbits=8)

#Load weight

In [3]:
# Define weights' address range
ADDR_OFFSET = 16

FC1_FROM = OFF_MEM_WB_BASE_ADDR
FC1_TO   = FC1_FROM + 0x0001_8800 - ADDR_OFFSET
print(f"FC1_FROM:{tohex(FC1_FROM, 32)}, FC1_TO:{tohex(FC1_TO, 32)}")
FC2_FROM = FC1_TO + ADDR_OFFSET
FC2_TO   = FC2_FROM + 0x0000_2000 - ADDR_OFFSET
print(f"FC2_FROM:{tohex(FC2_FROM,32)}, FC2_TO:{tohex(FC2_TO,32)}")
FC3_FROM = FC2_TO + ADDR_OFFSET
FC3_TO   = FC3_FROM + 0x0000_0280 - ADDR_OFFSET
print(f"FC3_FROM:{tohex(FC3_FROM,32)}, FC3_TO:{tohex(FC3_TO,32)}")

UB_FROM = OFF_MEM_UB_BASE_ADDR
UB_TO   = UB_FROM + 0x0000_0310 - ADDR_OFFSET
print(f"UB_FROM:{tohex(UB_FROM,32)}, UB_TO:{tohex(UB_TO,32)}")

FC1_FROM:00000000, FC1_TO:000187f0
FC2_FROM:00018800, FC2_TO:0001a7f0
FC3_FROM:0001a800, FC3_TO:0001aa70
UB_FROM:0001aa80, UB_TO:0001ad80


In [16]:
# 1. Prepare off-mem file(.coe)
# FC1
print(np.array(QM_DATA['weight']['FC1']))
FC1_weight = np.array(QM_DATA['weight']['FC1'])
for i in range(FC1_FROM, FC1_TO, 16):
    OFF_MEM.write(addr=i, val=encode(data=FC1_weight.tolist(), data_num=16, nbits=8))

# FC2


# FC3

[[-1 -1 -1 ... -1  0 -1]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  1  1]
 [ 0  1  0 ...  0  1  1]
 [-1 -1 -1 ... -1  0 -1]]


TypeError: '<' not supported between instances of 'list' and 'int'

In [5]:
# Write weight to WB
print("Write weight to WB")
for i in range(0, 256, 1):
    #weight = encode([- i + j for j in range(15, -1, -1)], 16, 8)
    #SA.WRITE_WEIGHT(i, weight)
    SA.AXI_TO_WB_INST(OFF_MEM, i, 256-i-1)
    SA.GENERATE_ISA("AXI_TO_WB_INST", i*16, (256-i-1)*16)
data = SA.WB_PRINT(dec=decode, do_print=DO_PRINT)
save_data("2_WB.txt", data)

Write weight to WB


NameError: name 'save_data' is not defined